In [1]:
import sys
import polars as pl
import boto3
import time
from IPython.display import display
import datetime
import math
from pathlib import PurePosixPath
print(f"{sys.version=}")

sys.version='3.14.0 (main, Oct  7 2025, 09:34:52) [Clang 17.0.0 (clang-1700.3.19.1)]'


# Список silver S3

In [2]:
s3 = boto3.resource(
    's3',
    endpoint_url='http://localhost:9000',
    aws_access_key_id='minioadmin',
    aws_secret_access_key='minioadmin',
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False
)
storage_options = {
    "aws_endpoint_url": "http://localhost:9000", # Важно: aws_endpoint_url (иногда endpoint_url)
    "aws_access_key_id": "minioadmin",
    "aws_secret_access_key": "minioadmin",
    "aws_region": "us-east-1", # Для MinIO часто можно оставить us-east-1
    "aws_allow_http": "true",  # Разрешить HTTP (без SSL)
}

In [3]:
bucket = s3.Bucket('silver')

In [8]:
def silver_path2dt(obj):
    return datetime.datetime(**{item.split("=",1)[0]: int(item.split("=",1)[1]) for item in PurePosixPath(obj.key).parts if "=" in item})
objects = sorted(bucket.objects.all(), key=silver_path2dt)

In [9]:
objects[0], objects[1]

(s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=12/part-bba9b6b6-a3f5-4165-bbd9-b34ab9a7fbe6.parquet'),
 s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=13/part-e374b7d5-01a4-49c0-b515-84e86c8283cd.parquet'))

In [10]:
for obj in objects:
    key = obj.key
    size_mb = obj.size//1e+6
    print(key, size_mb, obj.last_modified)

hh/vacancies/year=2025/month=11/day=12/part-bba9b6b6-a3f5-4165-bbd9-b34ab9a7fbe6.parquet 2.0 2025-12-12 09:04:06.609000+00:00
hh/vacancies/year=2025/month=11/day=13/part-e374b7d5-01a4-49c0-b515-84e86c8283cd.parquet 4.0 2025-12-12 09:04:38.370000+00:00
hh/vacancies/year=2025/month=11/day=14/part-c6077654-fe00-4dbe-8e13-9bc4e0f56fa8.parquet 4.0 2025-12-12 09:05:07.196000+00:00
hh/vacancies/year=2025/month=11/day=15/part-5157769f-7a83-429a-bb82-1bdef8d967f7.parquet 1.0 2025-12-12 09:05:16.560000+00:00
hh/vacancies/year=2025/month=11/day=16/part-dd809784-9f1d-49b5-a2f3-65d8b023a6b2.parquet 0.0 2025-12-12 09:05:23.998000+00:00
hh/vacancies/year=2025/month=11/day=17/part-aaa5d138-e5fa-4d3b-8a81-151e2e7e774b.parquet 7.0 2025-12-12 09:06:17.416000+00:00
hh/vacancies/year=2025/month=11/day=18/part-7e1d1cdf-0933-4c15-9ee3-c2b62f0d5681.parquet 6.0 2025-12-12 09:07:02.054000+00:00
hh/vacancies/year=2025/month=11/day=19/part-1471860e-073d-4cdc-bb43-7ed3639cb269.parquet 6.0 2025-12-12 09:07:42.21600

In [11]:
# objects = [obj for obj in objects if silver_path2dt(obj).day >8 and silver_path2dt(obj).month == 12]
# objects

# Общее кол-во вакансий в S3

In [12]:
for obj in objects:
    schema = pl.scan_parquet(f"s3://silver/{obj.key}",storage_options=storage_options).collect_schema()
    print(obj, len(schema))

s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=12/part-bba9b6b6-a3f5-4165-bbd9-b34ab9a7fbe6.parquet') 61
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=13/part-e374b7d5-01a4-49c0-b515-84e86c8283cd.parquet') 61
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=14/part-c6077654-fe00-4dbe-8e13-9bc4e0f56fa8.parquet') 61
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=15/part-5157769f-7a83-429a-bb82-1bdef8d967f7.parquet') 61
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=16/part-dd809784-9f1d-49b5-a2f3-65d8b023a6b2.parquet') 61
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=17/part-aaa5d138-e5fa-4d3b-8a81-151e2e7e774b.parquet') 61
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=18/part-7e1d1cdf-0933-4c15-9ee3-c2b62f0d5681.parquet') 61
s3.ObjectSummary(bucket_name='silv

In [13]:
df_lazy = pl.scan_parquet(
    [f"s3://silver/{obj.key}" for obj in objects],
    storage_options=storage_options,
)

In [14]:
df_lazy.select(
    pl.col('id').count().alias('id_cnt'),
    pl.col('id').unique().count().alias('unique_id_cnt')
).collect()

id_cnt,unique_id_cnt
u32,u32
689224,689224


In [25]:
df_lazy.tail(1).collect().glimpse()

Rows: 1
Columns: 61
$ id                                    <str> '128392992'
$ name                                  <str> 'Бетонщик (Забайкальский край)'
$ description                           <str> null
$ branded_description                   <str> null
$ key_skills                <list[struct[1]]> null
$ schedule                        <struct[2]> {'id': 'flyInFlyOut', 'name': 'Вахтовый метод'}
$ accept_handicapped                   <bool> null
$ accept_kids                          <bool> null
$ experience                      <struct[2]> {'id': 'between1And3', 'name': 'От 1 года до 3 лет'}
$ address                        <struct[10]> {'city': 'посёлок городского типа Новая Чара', 'street': None, 'building': None, 'lat': 56.79906, 'lng': 118.27571, 'description': None, 'raw': 'посёлок городского типа Новая Чара', 'metro': None, 'metro_stations': [], 'id': '19428704'}
$ employer                        <struct[9]> {'id': '11875449', 'name': 'ПрессБук', 'url': 'https://api.hh.ru/em